In [13]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [14]:
# write your code for task 0 here

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,False,39
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,False,35
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,False,27
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,False,43
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,False,25
...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,False,36
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,True,173
32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,False,40
32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,False,38


# Task 1
Print the count of men and women in the dataset.

In [15]:
# write your code for task 1 here

sex
Male      20380
Female     9782
Name: count, dtype: int64

# Task 2
Find the average age of men in dataset

In [16]:
# write your code for task 2 here

39.184

# Task 3
Get the percentage of people from Poland (native-country)

In [17]:
# write your code for task 3 here

0.1857

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [18]:
# write your code for task 4 here

mean less then 50: 36.6081
std less then 50: 13.4646
mean greater then 50: 43.9591
std greater then 50: 10.2696


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [19]:
# write your code for task 5 here

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,False,27
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,False,43
6,49,Private,9th,Married-spouse-absent,Other-service,Not-in-family,Black,Female,16,Jamaica,False,49
7,52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,True,307
10,37,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,True,116
...,...,...,...,...,...,...,...,...,...,...,...,...
32555,22,Private,Some-college,Never-married,Protective-serv,Not-in-family,White,Male,40,United-States,False,32
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,True,173
32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,False,40
32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,False,38


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [20]:
# write your code for task 6 here

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [21]:
# write your code for task 7 here

Married mean salary: 
salary
False     31.927538
True     200.609820
Name: salary K$, dtype: float64

Non married mean salary: 
salary
False     32.091316
True     196.010401
Name: salary K$, dtype: float64


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [22]:
# write your code for task 8 here

78

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [23]:
# write your code for task 9 here

Correlation matrix: 
                                     age  hours-per-week  salary K$  \
age                             1.000000        0.101599   0.208203   
hours-per-week                  0.101599        1.000000   0.196378   
salary K$                       0.208203        0.196378   1.000000   
workclass_Local-gov             0.068256        0.001612   0.025856   
workclass_Private              -0.210491       -0.095533  -0.098187   
...                                  ...             ...        ...   
native-country_Trinadad&Tobago  0.007868       -0.007566  -0.003442   
native-country_United-States    0.016259        0.010673   0.034000   
native-country_Vietnam         -0.017775       -0.010381  -0.014737   
native-country_Yugoslavia       0.000657        0.006983   0.008560   
salary_True                     0.241998        0.229480   0.853894   

                                workclass_Local-gov  workclass_Private  \
age                                        0.068256 